In [29]:
# Import the required libraries
import hvplot.pandas
import pandas as pd
import requests
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
from PIL import Image


# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import the GeoAPIFy Key
from config_api_keys import geo_api_key

In [2]:
# Read CSV
final_combined_happiness_csv = pd.read_csv('resources/final_mean_df.csv')
final_combined_happiness_csv

,Unnamed: 0,Country Name,Country Code,Region,Happiness Score,Life Expectancy,Suicide Rate,Unemployment Rate,Inflation,Net National Income Total ($) billions,GDP Total ($) billions
0,0,Albania,ALB,Europe,5.199,79.003400,4.60,14.0000,1.726728,10.95,13.37
1,1,Algeria,DZA,Africa,5.122,75.927400,2.56,10.5352,4.599003,134.99,168.56
2,2,Armenia,ARM,Europe,5.399,74.901800,4.06,18.1700,1.452263,10.39,11.74
3,3,Australia,AUS,Asia,7.162,82.599512,12.68,5.5620,1.651235,1040.77,1340.84
4,4,Austria,AUT,Europe,7.163,81.612683,15.44,5.3140,1.479740,340.09,418.94
...,...,...,...,...,...,...,...,...,...,...,...
106,106,Ukraine,UKR,Europe,5.084,71.571366,21.42,8.9960,19.177894,103.15,116.25
107,107,United States,USA,Americas,6.977,78.639024,15.44,4.4160,1.553023,16846.90,19658.50
108,108,Uruguay,URY,Americas,6.474,77.559600,20.58,8.0880,8.002460,48.04,60.10
109,109,Vietnam,VNM,Asia,5.485,73.969200,7.64,1.7540,2.631032,178.12,284.44


In [3]:
# Create a Data Frame
final_combined_happiness_df = pd.DataFrame(final_combined_happiness_csv)

final_combined_happiness_df.head()

,Unnamed: 0,Country Name,Country Code,Region,Happiness Score,Life Expectancy,Suicide Rate,Unemployment Rate,Inflation,Net National Income Total ($) billions,GDP Total ($) billions
0,0,Albania,ALB,Europe,5.199,79.003400,4.60,14.0000,1.726728,10.95,13.37
1,1,Algeria,DZA,Africa,5.122,75.927400,2.56,10.5352,4.599003,134.99,168.56
2,2,Armenia,ARM,Europe,5.399,74.901800,4.06,18.1700,1.452263,10.39,11.74
3,3,Australia,AUS,Asia,7.162,82.599512,12.68,5.5620,1.651235,1040.77,1340.84
4,4,Austria,AUT,Europe,7.163,81.612683,15.44,5.3140,1.479740,340.09,418.94


In [4]:
# Reduce the columns in the Data Frame
final_combined_happiness_reduced = final_combined_happiness_df[['Country Name',
                                                               'Region',
                                                               'Happiness Score',
                                                               'Suicide Rate']]

In [5]:
final_combined_happiness_reduced.head()

,Country Name,Region,Happiness Score,Suicide Rate
0,Albania,Europe,5.199,4.60
1,Algeria,Africa,5.122,2.56
2,Armenia,Europe,5.399,4.06
3,Australia,Asia,7.162,12.68
4,Austria,Europe,7.163,15.44


In [6]:
# final_combined_happiness_reduced['Location'] = None

# for index, row in final_combined_happiness_reduced.iterrows():
#     country = row['Country Name']
#     api_key = geo_api_key
#     url = f'https://api.geoapify.com/v1/geocode/search?text={country}&limit=1&apiKey={api_key}'
#     response = requests.get(url)
#     data = response.json()
#     location = data['features'][0]['geometry']['coordinates']
#     final_combined_happiness_reduced.at[index, 'location'] = location

In [7]:
# DF of the countries with 10 highest Happiness Score
final_combined_happiness_sorted = final_combined_happiness_reduced.sort_values(by='Happiness Score',ascending=False).iloc[:10,:]
final_combined_happiness_sorted

,Country Name,Region,Happiness Score,Suicide Rate
34,Finland,Europe,7.821,15.32
26,Denmark,Europe,7.636,11.04
100,Switzerland,Europe,7.512,14.54
74,Netherlands,Europe,7.415,11.80
61,Luxembourg,Europe,7.404,10.82
99,Sweden,Europe,7.384,14.98
80,Norway,Europe,7.365,12.20
49,Israel,Asia,7.364,5.10
75,New Zealand,Asia,7.200,11.80
4,Austria,Europe,7.163,15.44


In [8]:
# DF of the countries with 10 highest Suicide Rate
final_combined_suicide_sorted = final_combined_happiness_reduced.sort_values(by='Suicide Rate',ascending=False).iloc[:10,:]
final_combined_suicide_sorted

,Country Name,Region,Happiness Score,Suicide Rate
59,Lesotho,Africa,3.512,81.34
32,Eswatini,Africa,4.396,31.24
60,Lithuania,Europe,6.446,30.18
89,Russian Federation,Europe,5.459,28.68
96,South Africa,Africa,5.194,24.34
8,Belarus,Europe,5.821,23.16
106,Ukraine,Europe,5.084,21.42
108,Uruguay,Americas,6.474,20.58
58,Latvia,Europe,6.180,20.50
54,Kazakhstan,Asia,6.234,20.46


In [9]:
# -----------------------------
# Retrive the coordinate of the final_combined_happiness_sorted data frame

In [10]:
# New columns for Latitude and Longitude
final_combined_happiness_sorted['Latitude'] = ''
final_combined_happiness_sorted['Longitude'] = ''

In [11]:

for index, row in final_combined_happiness_sorted.iterrows():
    country = row['Country Name']
    url = f'https://api.geoapify.com/v1/geocode/search?text={country}&limit=1&apiKey={geo_api_key}'
    response = requests.get(url)
    data = response.json()
    try:
        latitude = data['features'][0]['geometry']['coordinates'][1]
        longitude = data['features'][0]['geometry']['coordinates'][0]
        final_combined_happiness_sorted.at[index, 'Latitude'] = latitude
        final_combined_happiness_sorted.at[index, 'Longitude'] = longitude
    except (IndexError, KeyError):
        # In case the API returns no data or the JSON structure is different
        pass

In [12]:
final_combined_happiness_sorted = final_combined_happiness_sorted.reset_index(drop=True)
final_combined_happiness_sorted.index = final_combined_happiness_sorted.index + 1
final_combined_happiness_sorted

,Country Name,Region,Happiness Score,Suicide Rate,Latitude,Longitude
1,Finland,Europe,7.821,15.32,63.246778,25.920916
2,Denmark,Europe,7.636,11.04,55.670249,10.333328
3,Switzerland,Europe,7.512,14.54,46.798562,8.231974
4,Netherlands,Europe,7.415,11.80,52.243498,5.634323
5,Luxembourg,Europe,7.404,10.82,49.815868,6.129675
6,Sweden,Europe,7.384,14.98,59.674971,14.520858
7,Norway,Europe,7.365,12.20,61.152939,8.787665
8,Israel,Asia,7.364,5.10,30.812425,34.859476
9,New Zealand,Asia,7.200,11.80,-41.500083,172.834408
10,Austria,Europe,7.163,15.44,47.59397,14.12456


In [13]:
happiness_map_plot = final_combined_happiness_sorted.hvplot.points(
    x = "Longitude",
    y = "Latitude",
    geo = True,
    title = 'Top 10 Countries With Highest Happiness Score',
    tiles = "OSM",
    hover_cols = ["Country Name", "Region", "Happiness Score"],
    size = "Happiness Score",
    scale = 4,
    frame_width = 700,
    frame_height = 500,
    color = ['Country Name','Happiness Score'],
    hover_line_color = "Happiness Score",
    hover_line_width = 2,
)

# Display the map plot
happiness_map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Happiness Score,Country Name,Region)

In [36]:
# Save the map plot as a PNG file
png_save_path = 'happiness_map_plot.png'
fig = happiness_map_plot.options(toolbar=None).opts(width=700, height=500).to.plot()
fig.savefig(png_save_path)


AttributeError: 'Overlay' object has no attribute to.

In [14]:
# -----------------------------
# Retrive the coordinate of the final_combined_happiness_sorted data frame

In [15]:
# New columns for Latitude and Longitude
final_combined_suicide_sorted['Latitude'] = ''
final_combined_suicide_sorted['Longitude'] = ''

In [16]:
for index, row in final_combined_suicide_sorted.iterrows():
    country = row['Country Name']
    url = f'https://api.geoapify.com/v1/geocode/search?text={country}&limit=1&apiKey={geo_api_key}'
    response = requests.get(url)
    data = response.json()
    try:
        latitude = data['features'][0]['geometry']['coordinates'][1]
        longitude = data['features'][0]['geometry']['coordinates'][0]
        final_combined_suicide_sorted.at[index, 'Latitude'] = latitude
        final_combined_suicide_sorted.at[index, 'Longitude'] = longitude
    except (IndexError, KeyError):
        # In case the API returns no data or the JSON structure is different
        pass

In [17]:
final_combined_suicide_sorted = final_combined_suicide_sorted.reset_index(drop=True)
final_combined_suicide_sorted.index = final_combined_suicide_sorted.index + 1
final_combined_suicide_sorted

,Country Name,Region,Happiness Score,Suicide Rate,Latitude,Longitude
1,Lesotho,Africa,3.512,81.34,-29.603927,28.335019
2,Eswatini,Africa,4.396,31.24,-26.562481,31.399132
3,Lithuania,Europe,6.446,30.18,55.35,23.75
4,Russian Federation,Europe,5.459,28.68,64.686314,97.745306
5,South Africa,Africa,5.194,24.34,-28.816624,24.991639
6,Belarus,Europe,5.821,23.16,53.425061,27.697136
7,Ukraine,Europe,5.084,21.42,49.487197,31.271832
8,Uruguay,Americas,6.474,20.58,-32.875555,-56.020153
9,Latvia,Europe,6.180,20.50,56.840649,24.753764
10,Kazakhstan,Asia,6.234,20.46,48.101295,66.778082


In [18]:
# Creat HVplot of suicide rate
suicide_map_plot = final_combined_suicide_sorted.hvplot.points(
    x = "Longitude",
    y = "Latitude",
    geo = True,
    title = '10 Countries With Highest Suicide Rate',
    tiles = "OSM",
    size = "Suicide Rate",
    scale = 2,
    frame_width = 700,
    frame_height = 500,
    color = ["Country Name"],
    hover_cols = ["Country Name", "Region"],
    hover_line_color = "Suicide Rate",
    hover_line_width = 2
)


# Display the map plot
suicide_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Suicide Rate,Country Name,Region)

In [19]:
# -------------------------------
# Full World Happiness Score Map

In [20]:
# Ordered Happiness Score
world_happiness_sorted = final_combined_happiness_reduced.sort_values(by='Happiness Score',ascending=False)
world_happiness_sorted

,Country Name,Region,Happiness Score,Suicide Rate
34,Finland,Europe,7.821,15.32
26,Denmark,Europe,7.636,11.04
100,Switzerland,Europe,7.512,14.54
74,Netherlands,Europe,7.415,11.80
61,Luxembourg,Europe,7.404,10.82
...,...,...,...,...
63,Malawi,Africa,3.750,5.84
101,Tanzania,Africa,3.702,4.22
59,Lesotho,Africa,3.512,81.34
12,Botswana,Africa,3.471,17.54


In [21]:
# New columns for Latitude and Longitude
world_happiness_sorted['Latitude'] = ''
world_happiness_sorted['Longitude'] = ''

In [22]:
# Retrive the coordinate of happiness score for all countries
for index, row in world_happiness_sorted.iterrows():
    country = row['Country Name']
    url = f'https://api.geoapify.com/v1/geocode/search?text={country}&limit=1&apiKey={geo_api_key}'
    response = requests.get(url)
    data = response.json()
    try:
        latitude = data['features'][0]['geometry']['coordinates'][1]
        longitude = data['features'][0]['geometry']['coordinates'][0]
        world_happiness_sorted.at[index, 'Latitude'] = latitude
        world_happiness_sorted.at[index, 'Longitude'] = longitude
    except (IndexError, KeyError):
        # In case the API returns no data or the JSON structure is different
        pass

In [23]:
world_happiness_sorted = world_happiness_sorted.reset_index(drop=True)
world_happiness_sorted.index = world_happiness_sorted.index + 1
world_happiness_sorted

,Country Name,Region,Happiness Score,Suicide Rate,Latitude,Longitude
1,Finland,Europe,7.821,15.32,63.246778,25.920916
2,Denmark,Europe,7.636,11.04,55.670249,10.333328
3,Switzerland,Europe,7.512,14.54,46.798562,8.231974
4,Netherlands,Europe,7.415,11.80,52.243498,5.634323
5,Luxembourg,Europe,7.404,10.82,49.815868,6.129675
...,...,...,...,...,...,...
107,Malawi,Africa,3.750,5.84,-13.26872,33.930196
108,Tanzania,Africa,3.702,4.22,-6.524712,35.787844
109,Lesotho,Africa,3.512,81.34,-29.603927,28.335019
110,Botswana,Africa,3.471,17.54,-23.168178,24.592874


In [37]:
# Use plotly.express to create a world happiness score map
world_happiness_map = px.choropleth(world_happiness_sorted, 
                          locations = 'Country Name', 
                          locationmode='country names', 
                          color='Happiness Score',
                          color_continuous_scale="Viridis",
                          title = 'World Happiness Score'
                          )
                         
world_happiness_map.update_layout(margin={"r":0,"t":10,"l":0,"b":0},
                                  title_x=0.45, 
                                  title_y=0.90,
                                  width=900,
                                  height= 500
                                 )

world_happiness_map.show()

In [28]:
world_happiness_map.write_html('world_happiness_map.html')

In [ ]:
world_happiness_map.w